# 1. Problema de Transporte

El objetivo del problema es minimizar el costo total de envío a todos los clientes de todas las fuentes.
$$
\text { minimizar: } \quad \text { Costo }=\sum_{c \in \text { Clientes }} \sum_{s \in \text { Fuentes }} T[c, s] x[c, s]
$$
Los envíos de todas las fuentes no pueden exceder la capacidad de fabricación de la fuente.
$$
\sum_{c \in \text { Customers }} x[c, s] \leq \text { Oferta }[s] \quad \forall s \in \text { Fuentes }
$$
Los envíos a cada cliente deben satisfacer su demanda.
$$
\sum_{s \in \text { Fuentes }} x[c, s]=\text { Demanda }[c] \quad \forall c \in \text { Clientes }
$$

## Intalar pyomo
```python
pip install pyomo
```

In [2]:
pip install pyomo

     |████████████████████████████████| 5.6 MB 9.4 MB/s eta 0:00:01
  Using cached ply-3.11-py2.py3-none-any.whl (49 kB)
Note: you may need to restart the kernel to use updated packages.


## Paso 2 - Intalar el solver
```
!apt-get install -y -qq coinor-cbc
```


In [13]:
!apt-get install -y -qq coinor-cbc

zsh:1: command not found: apt-get


## Ahora escribimos nuestro programa en pyomo

```
import pandas
from pyomo.environ import *
df = pandas.read_excel('wl_data.xlsx', 'Hoja1', header=0, index_col=0)
O = list(df.index.map(str))
D = list(df.columns.map(str))
d = {(r, c):df.at[r,c] for r in O for c in D}
Demanda = {'Bucaramanga':125,    
   'Cartagena':   175,        
   'Buga':   225,        
   'Pereira':   250}

Oferta = {
   'Bogota':   300,        
   'Cali':   350,
   'Medellin': 125
}
model = ConcreteModel()
model.x = Var(O, D, domain = NonNegativeReals)
model.obj = Objective(expr = sum(d[o,de]*model.x[o,de] for o in O for de in D),sense=minimize)

model.o = ConstraintList()
for o in O:
    model.o.add(sum([model.x[o,de] for de in D]) == Oferta[o])

model.de = ConstraintList()
for de in D:
    model.de.add(sum([model.x[o,de] for o in O]) == Demanda[de])
    
results = SolverFactory('cbc').solve(model).write()

for c in O:
    for s in D:
        print(c, s, model.x[c,s]())
```


In [14]:
import pandas as pd
from pyomo.environ import *
import os
os.environ['NEOS_EMAIL'] = 'jorgei.romerog@utadeo.edu.co'

costos=pd.read_excel('/Users/jorgeiv500/Dropbox/Mac/Documents/Logistica/Transporte/tl.xlsx','Hoja1',header=0,index_col=0)
costos

,Bucaramanga,Cartagena,Buga,Pereira
Bogota,1956,1606,1410,330
Cali,1096,1792,531,567
Medellin,485,2322,324,1236


In [15]:
O=list(costos.index.map(str))
print(O)
D=list(costos.columns.map(str))
print(D)

['Bogota', 'Cali', 'Medellin']
['Bucaramanga', 'Cartagena', 'Buga', 'Pereira']


In [16]:
d = {(i,j):costos.at[i,j] for i in O for j in D}
d

{('Bogota', 'Bucaramanga'): 1956,
 ('Bogota', 'Cartagena'): 1606,
 ('Bogota', 'Buga'): 1410,
 ('Bogota', 'Pereira'): 330,
 ('Cali', 'Bucaramanga'): 1096,
 ('Cali', 'Cartagena'): 1792,
 ('Cali', 'Buga'): 531,
 ('Cali', 'Pereira'): 567,
 ('Medellin', 'Bucaramanga'): 485,
 ('Medellin', 'Cartagena'): 2322,
 ('Medellin', 'Buga'): 324,
 ('Medellin', 'Pereira'): 1236}

In [17]:
Demanda = {'Bucaramanga':125,'Cartagena': 175,'Buga': 225,'Pereira':   250}
print(Demanda)
Oferta = {'Bogota':300,'Cali':350,'Medellin': 125 }
print(Oferta)

{'Bucaramanga': 125, 'Cartagena': 175, 'Buga': 225, 'Pereira': 250}
{'Bogota': 300, 'Cali': 350, 'Medellin': 125}


In [19]:
transporte= ConcreteModel()
transporte.x=Var(O,D,domain=NonNegativeReals)
transporte.obj=Objective(expr=sum(d[i,j]*transporte.x[i,j] for i in O for j in D),sense=minimize)

transporte.ro = ConstraintList() # restriccion oferta
for i in O:
  transporte.ro.add(sum([transporte.x[i,j] for j in D])==Oferta[i])

transporte.rd = ConstraintList() # restriccion demanda
for j in D:
  transporte.rd.add(sum([transporte.x[i,j] for i in O])== Demanda[j])

solver_manager = SolverManagerFactory('neos')
resultados = solver_manager.solve(transporte, solver='cplex')

minimo = transporte.obj()
print('El minimo costo es = ', minimo)

for i in O:
  for j in D:
    print(i,j,transporte.x[i,j]())

El minimo costo es =  566900.0
Bogota Bucaramanga 0.0
Bogota Cartagena 50.0
Bogota Buga 0.0
Bogota Pereira 250.0
Cali Bucaramanga 0.0
Cali Cartagena 125.0
Cali Buga 225.0
Cali Pereira 0.0
Medellin Bucaramanga 125.0
Medellin Cartagena 0.0
Medellin Buga 0.0
Medellin Pereira 0.0
